In [ ]:
# 将 llm_toy/src 加入 sys.path（兼容多启动位置）
import sys
from pathlib import Path

def _add_src_path():
    candidates = [
        Path.cwd() / 'llm_toy' / 'src',           # 在项目根启动Jupyter
        Path.cwd() / 'src',                        # 在 llm_toy 目录启动
        Path.cwd().parent / 'llm_toy' / 'src',
        Path.cwd().parent / 'src',
    ]
    # 向上回溯几层尝试
    for base in list(Path.cwd().parents)[:3]:
        candidates.append(base / 'llm_toy' / 'src')
        candidates.append(base / 'src')
    for p in candidates:
        if (p / 'model.py').exists() and (p / 'utils.py').exists():
            sys.path.append(str(p.resolve()))
            print('已添加src路径:', p.resolve())
            return str(p.resolve())
    print('警告：未找到 llm_toy/src，请手动添加路径或调整工作目录。')
    return None

SRC_PATH = _add_src_path()


# 05 Attention可视化：观察模型关注了哪些Token

本Notebook展示如何从 GPT-2 提取Attention权重，并用heatmap进行可视化。

重点概念：Self-Attention、Heads、Layers、Attention Map。

In [ ]:
import os
from pathlib import Path
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import sys
# 路径已在开头插入cell中处理
from model import SimpleGPTModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


## 准备模型与输入

提示：本例不训练，仅做前向并拿到attention。

In [ ]:
simple = SimpleGPTModel(model_name='gpt2')
tokenizer = simple.tokenizer
model = simple.model.to(device)

text = 'Attention is all you need, 也是Transformer的核心之一。'
enc = tokenizer(text, return_tensors='pt')
enc = {k: v.to(device) for k, v in enc.items()}

with torch.no_grad():
    outputs = model(**enc, output_attentions=True)
attentions = outputs.attentions  # List[layer] -> (batch, heads, seq, seq)
len(attentions), [a.shape for a in attentions]


## 可视化最后一层的平均Attention

- 取最后一层，对所有heads取平均
- x/y轴使用对应的tokens（注意BPE切分）


In [ ]:
last = attentions[-1][0]  # (heads, seq, seq) 取batch维0
avg_attn = last.mean(dim=0).detach().cpu().numpy()  # (seq, seq)
tokens = tokenizer.convert_ids_to_tokens(enc['input_ids'][0])
plt.figure(figsize=(8,6))
sns.heatmap(avg_attn, xticklabels=tokens, yticklabels=tokens, cmap='viridis')
plt.title('Last Layer Avg Attention')
plt.xlabel('Key tokens')
plt.ylabel('Query tokens')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 练习

- 观察不同layer、不同head的Attention差异
- 尝试不同输入句子，比较中英文token化对可视化的影响
- 对生成过程中每一步取Attention（需要使用past_key_values与步进解码）
